# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import geopandas as gpd
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import requests

# Import API key
# from api_keys import geoapify_key
from dotenv import load_dotenv
import os

load_dotenv()
geoapify_key = os.getenv("GEOAPIFY_KEY")

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,33.91,96,100,2.53,CL,1724548461
1,1,yangju,37.8331,127.0617,84.07,66,0,0.09,KR,1724548462
2,2,mossel bay,-34.1831,22.1460,54.09,74,39,2.24,ZA,1724548463
3,3,grytviken,-54.2811,-36.5092,25.09,78,14,11.36,GS,1724548464
4,4,waitangi,-43.9535,-176.5597,54.82,97,100,5.99,NZ,1724548465


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
lon = city_data_df["Lng"]
lat = city_data_df["Lat"]

# Create a base map
base_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Max Temp",
    cmap="coolwarm",
    colorbar=True,
    title="City Latitude vs. Longitude",
    xlabel="Longitude",
    ylabel="Latitude",
    frame_height=600,
    frame_width=800,
    grid=True,
    size= "Humidity",
    geo=True,
    color = "City"
    

)

# Display the map
base_map





:Points   [Lng,Lat]   (City,Humidity)

In [37]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_temp_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= 90) & (city_data_df["Max Temp"] >= 40)
    & (city_data_df["Humidity"] < 50) & (city_data_df["Humidity"] > 20)
    & (city_data_df["Wind Speed"] <= 10)
    & (city_data_df["Cloudiness"] <= 20)
    
].dropna()

# Display sample data
ideal_temp_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
191,191,yanykurgan,43.9150,67.2481,85.69,21,0,5.50,KZ,1723388702
224,224,embi,48.8264,58.1442,79.05,32,15,9.86,KZ,1723388742
250,250,fabens,31.5023,-106.1586,84.06,32,0,8.48,US,1723388773
298,298,madaba,31.5833,35.6667,87.91,44,0,5.08,JO,1723388831
306,306,santa isabel,-23.3156,-46.2214,66.74,35,6,2.91,BR,1723388842
323,323,mount isa,-20.7333,139.5000,62.37,42,2,6.91,AU,1723388862
364,364,mount carmel,38.4109,-87.7614,72.14,49,20,6.93,US,1723388950
404,404,mwanza,-2.5167,32.9000,82.44,38,5,7.20,TZ,1723388998
448,448,placido de castro,-10.3353,-67.1856,84.87,26,0,2.64,BR,1723389052
457,457,ariquemes,-9.9133,-63.0408,89.26,29,2,2.73,BR,1723389063


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,puerto natales,CL,-51.7236,-72.4875,86,
8,grytviken,GS,-54.2811,-36.5092,58,
13,qaqortoq,GL,60.7167,-46.0333,89,
17,port-aux-francais,TF,-49.3500,70.2167,85,
28,ushuaia,AR,-54.8000,-68.3000,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
limit = 10
category = "accommodation.hotel"

# Set the parameters for the Geoapify API
params = {
    "limit": limit,
    "categories": category,
    "apiKey": geoapify_key,
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL

    base_url = "https://api.geoapify.com/v2/places"


    
    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



# Display sample data
hotel_df

Starting hotel search
puerto natales - nearest hotel: Dorotea Patagonia Hostel
grytviken - nearest hotel: No hotel found
qaqortoq - nearest hotel: Hotel Qaqortoq
port-aux-francais - nearest hotel: Keravel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
tiksi - nearest hotel: Арктика
sarmiento - nearest hotel: Ismar
puerto aysen - nearest hotel: Hostal Dendal
bilibino - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
olonkinbyen - nearest hotel: No hotel found
las heras - nearest hotel: Posadas del Sur
kerikeri - nearest hotel: Avalon Resort
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
esquel - nearest hotel: El Quincho
aasiaat - nearest hotel: SØMA
xinyuan - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
whakatane - nearest hotel: Whakatane Hotel
masterton - nearest hotel: Copthorne Hotel Solway Park
colchani - nearest hotel: Hotel de Sal -

,City,Country,Lat,Lng,Humidity,Hotel Name
2,puerto natales,CL,-51.7236,-72.4875,86,Dorotea Patagonia Hostel
8,grytviken,GS,-54.2811,-36.5092,58,No hotel found
13,qaqortoq,GL,60.7167,-46.0333,89,Hotel Qaqortoq
17,port-aux-francais,TF,-49.3500,70.2167,85,Keravel
28,ushuaia,AR,-54.8000,-68.3000,80,Apart Hotel Aires del Beagle
32,tiksi,RU,71.6872,128.8694,100,Арктика
35,sarmiento,AR,-45.5882,-69.0700,66,Ismar
48,puerto aysen,CL,-45.4000,-72.7000,99,Hostal Dendal
62,bilibino,RU,68.0546,166.4372,99,No hotel found
66,punta arenas,CL,-53.1500,-70.9167,93,Hotel Lacolet


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    cmap="coolwarm",
    colorbar=True,
    title="Hotel Locations",
    xlabel="Longitude",
    ylabel="Latitude",
    frame_height=500,
    frame_width=700,
    grid=True,
    size= "Humidity",
    hover_cols=["City", "Country", "Hotel Name", "Humidity"],
    geo=True,
)

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)